Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `???` or "YOUR ANSWER HERE".

---

# CSCI 3155 Assignment 5 : Let Bindings and Scopes

This assignment asks you to write scala programs. 

**Restrictions** apply to each problem in terms of forbidden Scala features and API functions. Please read them carefully and ask for clarifications from the course staff over Piazza or during office hours if unsure.

Note: `???` indicates that there is a missing function or code fragment that needs to be filled in. In scala, 
it is also a macro that throws a `NotImplemented` exception. Make sure that you remove the `???` and replace it with the answer. 

Use the test cases provided to test them. You are also encouraged to write your own test cases to help debug your work. However, please delete any extra cells you may have created lest they break our autograder.

**Very Important:** Please run the cell that defines the functions `passed` and `testWithMessage` below whenever you restart the notebook.

### Your Name Here

In [1]:
// TEST HELPER

// FIRST RUN THIS CELL EVERY TIME YOU START THE NOTEBOOK
def passed(points: Int) {
    require(points >=0)
    if (points == 1) print(s"\n*** Tests Passed (1 point) ***\n")
    else print(s"\n*** Tests Passed ($points points) ***\n")
}

def testWithMessage[T](v1: T, expected: T, testID: String) = { 
    println(s"Test $testID"); 
    println(s"\t Your code returned: $v1, Expected: $expected")
    assert (v1 == expected, s"Test $testID FAILED.")
    println("\t Passed!")
}
/*

def testWithMessage(v1: Double, expected: Double, testID: String) = {
    val tolerance = 1E-5
    println(s"Test $testID -- comparing with tolerance $tolerance."); 
    println(s"\t Expected: $expected, your code returned: $v1")
    assert (math.abs(v1-expected) <= tolerance, s"Test $testID FAILED.")
    println("\t Passed!")
}
*/

defined function passed
defined function testWithMessage

## Problem 1: Multiple Simultaneous Let Bindings (15 points)

In class, we studied let bindings that assigned a "single" identifier to a "single" expression. Here, we will extend lettuce with multiple let bindings at the same time:

## Example 1 
~~~
let (x, y, z) = (10, 25.6, 30.3) in 
   x - y * z
~~~

The program computes `10 - 25.6 * 30.3`.

## Example 2

 In multi-let binding, we treat all the assignments as happening "simultaneously". For instance, the program 

~~~
let (x, y, z) = (10, x, y - x) in 
  x - y * z
~~~

is disallowed since neither `x` nor `y` are in scope in the right hand side of the multi-let binding. Example 2 above produces an `error` value since `x` and `y` are out of scope on the right hand side of the assignment.

## Example 3

~~~
let x = 15 in 
  let (x, y, z) = (x*x, -10 *x, -2*x) in 
     x + y + z
~~~

Note that the usage `x*x`, `-10*x` and `-2*x` refer back to `let x = 15` definition. However, the usages `x+y+z` refer to the result of the "multi-let" binding. Verify that this program will evaluate to "45".

## Grammar of Lettuce

Let us extend a minimalistic subset of Lettuce by adding a `MultiLet` statement as shown below.
$$\newcommand\Expr{\mathbf{Expr}}$$

$$\begin{array}{rcll}
  \Expr & \Rightarrow & \text{Const}(\mathbf{Double}) \\
  & | & \text{Ident}(\mathbf{String}) \\
  & | & \text{Plus}(\mathbf{Expr}, \mathbf{Expr}) \\
  & | & \text{Mult}(\mathbf{Expr}, \mathbf{Expr})\\
  & | & \text{Let}(\mathbf{Ident}, \mathbf{Expr}, \mathbf{Expr}) \\
  & | & \text{MultiLet}(\mathbf{Ident}*, \mathbf{Expr}*, \mathbf{Expr}) & \leftarrow\ \text{ let (x1, .., xn) = (e1, ...,en) in e } \\
  & & & \text{Note: Number of identifiers n must match number of expressions n, or else evaluate to error }\\
  \end{array}$$
  
The scala definitions are given below.

In [2]:
sealed trait Expr
case class Const(d: Double) extends Expr
case class Ident(s: String) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Mult(e1: Expr, e2: Expr) extends Expr 
case class Let(id: String, e1: Expr, e2: Expr) extends Expr
case class MultiLet(id: List[String], eList: List[Expr], e2: Expr) extends Expr

defined trait Expr
defined class Const
defined class Ident
defined class Plus
defined class Mult
defined class Let
defined class MultiLet

## Semantics for MultiLet

$$\newcommand\semrule[3]{\begin{array}{c} #1 \\ \hline #2 \\ \end{array}(\text{#3})} $$

Let us write down the semantic rules for a multilet statement:

$$\newcommand\eval{\textit{eval}}$$
$$\semrule{ \eval(\texttt{ei}, \texttt{env})= v_i,\ v_i \not= \mathbf{error}, \text{for}\ i = 1, \ldots, n,\ \texttt{newenv} = env \circ \{ \texttt{x1} \mapsto v_1, \ldots, \texttt{xn} \mapsto v_n \} }{ \eval( \texttt{MultiLet([x1,..,xn], [e1,...,en], eBody), env}) = \eval(\texttt{eBody, newenv})}{multilet-ok}$$

The semantic rule above tells you to 
  - Evaluate each of the expressions from `e1`, ..., `en` under the environment `env`.
  - Next, if all the expressions above evaluated without an error, it tells you to update the map `env` by binding each `xi` to $v_i$, the result of evaluating `ei`. You can use the Scala Map "++" operator to achieve this in one shot.
  
  ~~~
  val m1 : Map[String, Int] = Map( "x" -> 10, "y" -> 20, "z" -> 30 )
  val m2 : Map[String, Int] = Map( "w" -> 40, "l" -> 50, "z" -> 60)
  val m3 = m1 ++ m2 // Join the two maps together and obtain a new map. z will map to 60
  ~~~
  - Finally, you should evaluate `eBody` under the new environment created.

For convenience, we write a single "generic" semantic rule that shows that if some argument `ej` evaluates to an error, the whole expression is erroneous.

$$\semrule{ j \leq n, \eval(\texttt{ei}, \texttt{env})= v_i,\ v_i \not= \mathbf{error}, \text{for}\ i = 1, \ldots, j-1,\ \ eval(\texttt{ej}, \texttt{env})= \mathbf{error} }{ \eval( \texttt{MultiLet([x1,..,xn], [e1,...,en], eBody), env}) = \mathbf{error}}{multilet-err-j}$$

### Interpreter for MultiLet Statements

Implement an interpreter for the lettuce language with `multi-let` statements. Your interpreter does not need to "propagate" error: instead you should throw an `IllegalArgumentException` whenever an error is encountered. 

### Style Guide

Use of var/while/for loops in your solution below is disallowed.


In [3]:
sealed trait Value
case class NumValue(f: Double) extends Value
case object Error extends Value /* -- Do not return Error -- simply throw an new IllegalArgumentException whenever you encounter an erroneous case --*/

type Environment = Map[String, Value]

def evalExpr(e: Expr, env: Environment): Value = {
    
    e match {
        case Const(f) => NumValue(f)
        case Ident(x) => { 
            if (env.contains(x)) { 
                env(x)
            } else {
                throw new IllegalArgumentException("Not found identifier")
            }
        }
        case Plus(e1, e2) => {
            val v1 = evalExpr(e1, env)
            val v2 = evalExpr(e2, env)
            (v1, v2) match {
                case (NumValue(f1), NumValue(f2)) => NumValue(f1 + f2)
                case _ => throw new IllegalArgumentException("plus failed")
            }
        }
        case Mult(e1, e2) => {
            val v1 = evalExpr(e1, env)
            val v2 = evalExpr(e2, env)
            (v1, v2) match {
                case (NumValue(f1), NumValue(f2)) => NumValue(f1 * f2)
                case _ => throw new IllegalArgumentException("mult failed")
            }
        }
        case Let(x, e1, e2) => {
            // YOUR CODE HERE
            val v1 = evalExpr(e1, env);
            if (v1 == null)
                throw new IllegalArgumentException("Let Failed")
            else
                evalExpr(e2, env + (x -> v1))
            
        }
        case MultiLet(xList, eList, e2) => {
            if(xList.length != eList.length)
            {
                throw new IllegalArgumentException("MultiLet Failed")
            }
            else
            {
                /*
                    *This converts whatever the _ and env to Value and a combine it with eList
                    Zip the two list xList and ziplst(converted to a map) and combine env and xList
                    then change e2 and newEnv into eval.
                */
                val ziplst = eList.map(evalExpr(_,env))
                val newEnv = xList.zip(ziplst).toMap ++ env;
                evalExpr(e2, newEnv)
                
               
            }
        }
    }
   
}


defined trait Value
defined class NumValue
defined object Error
defined type Environment
defined function evalExpr

In [4]:
//BEGIN TEST
/*
 let (x, y) = (10, 20) in 
    let x = y in 
      x +  x * y
*/
val x = Ident("x")
val y = Ident("y")
val let1 = Let("x", y, Plus(x, Mult(x, y)) )
val mlet1 = MultiLet( List("x", "y"), List(Const(10.0), Const(20.0)), let1)
val v = evalExpr(mlet1, Map.empty)
assert(v == NumValue(420.0), s"Test 1 failed expected: NumValue(420.0), obtained $v")

passed(6)
//END TEST


*** Tests Passed (6 points) ***


x: Ident = Ident(s = "x")
y: Ident = Ident(s = "y")
let1: Let = Let(
  id = "x",
  e1 = Ident(s = "y"),
  e2 = Plus(
    e1 = Ident(s = "x"),
    e2 = Mult(e1 = Ident(s = "x"), e2 = Ident(s = "y"))
  )
)
mlet1: MultiLet = MultiLet(
  id = List("x", "y"),
  eList = List(Const(d = 10.0), Const(d = 20.0)),
  e2 = Let(
    id = "x",
    e1 = Ident(s = "y"),
    e2 = Plus(
      e1 = Ident(s = "x"),
      e2 = Mult(e1 = Ident(s = "x"), e2 = Ident(s = "y"))
    )
  )
)
v: Value = NumValue(f = 420.0)

In [5]:
//BEGIN TEST
/*
 let (x, y) = (10, x) in 
    let x = y in 
      x +  x * y
*/
val x = Ident("x")
val y = Ident("y")
val let1 = Let("x", y, Plus(x, Mult(x, y)) )
val mlet1 = MultiLet( List("x", "y"), List(Const(10.0), x), let1)
try {
    val v = evalExpr(mlet1, Map.empty)
    assert(false, "Test 2 failed -- your code should detect a usage of x that is out of scope")
} catch {
    case e:IllegalArgumentException => { println("Illegal argument exception caught -- as expected!!") }
    case _ => {println("Wrong type of exception thrown")}
}

passed(7)
//END TEST

Illegal argument exception caught -- as expected!!

*** Tests Passed (7 points) ***


x: Ident = Ident(s = "x")
y: Ident = Ident(s = "y")
let1: Let = Let(
  id = "x",
  e1 = Ident(s = "y"),
  e2 = Plus(
    e1 = Ident(s = "x"),
    e2 = Mult(e1 = Ident(s = "x"), e2 = Ident(s = "y"))
  )
)
mlet1: MultiLet = MultiLet(
  id = List("x", "y"),
  eList = List(Const(d = 10.0), Ident(s = "x")),
  e2 = Let(
    id = "x",
    e1 = Ident(s = "y"),
    e2 = Plus(
      e1 = Ident(s = "x"),
      e2 = Mult(e1 = Ident(s = "x"), e2 = Ident(s = "y"))
    )
  )
)

In [6]:
//BEGIN TEST
/*
let (x, y, z, w) = (10, 10, 10, 20 ) in 
  let () = () in 
    let w = w in 
       x *( y + w )
*/

val x = Ident("x")
val y = Ident("y")
val z = Ident("z")
val w = Ident("w")
val ten = Const(10.0)
val twenty = Const(20.0)
val innerLet2 = Let("w", w, Mult(x, Plus(y, w)))
val multiLet1 = MultiLet(Nil, Nil, innerLet2)
val e = MultiLet(List("x","y","z","w"), List(ten, ten, ten, twenty), multiLet1)
val v = evalExpr(e, Map.empty)
assert(v == NumValue(300.0), "Test2 Failed -- expected value NumValue(300.0), obtained value $v")

passed(8)
//END TEST


*** Tests Passed (8 points) ***


x: Ident = Ident(s = "x")
y: Ident = Ident(s = "y")
z: Ident = Ident(s = "z")
w: Ident = Ident(s = "w")
ten: Const = Const(d = 10.0)
twenty: Const = Const(d = 20.0)
innerLet2: Let = Let(
  id = "w",
  e1 = Ident(s = "w"),
  e2 = Mult(
    e1 = Ident(s = "x"),
    e2 = Plus(e1 = Ident(s = "y"), e2 = Ident(s = "w"))
  )
)
multiLet1: MultiLet = MultiLet(
  id = List(),
  eList = List(),
  e2 = Let(
    id = "w",
    e1 = Ident(s = "w"),
    e2 = Mult(
      e1 = Ident(s = "x"),
      e2 = Plus(e1 = Ident(s = "y"), e2 = Ident(s = "w"))
    )
  )
)
e: MultiLet = MultiLet(
  id = List("x", "y", "z", "w"),
  eList = List(
    Const(d = 10.0),
    Const(d = 10.0),
    Const(d = 10.0),
    Const(d = 20.0)
  ),
  e2 = MultiLet(
    id = List(),
    eList = List(),
    e2 = Let(
      id = "w",
      e1 = Ident(s = "w"),
      e2 = Mult(
        e1 = Ident(s = "x"),
        e2 = Plus(e1 = Ident(s = "y"), e2 = Ident(s = "w"))
      )
    )
  )
)
v: Value = NumValue(f = 300.0)

## Problem 2: Tag Usages of Identifiers with Definitions (35 points)

In this problem, we will extend Lettuce by adding user-defined "tags" to the `Let` bindings and usages `Ident` in the abstract syntax. The task is to produce a mapping from each ident-tag back to the tag of the let binding that defines the variable, such that scoping becomes explicit. This kind of analysis is called a "use-def" chain and is used in many compilers to optimize the code that is generated (https://en.wikipedia.org/wiki/Use-define_chain).

## Example 1

~~~
let x (@tag1) = 10 in 
   let y (@tag2 ) = ( let x (@tag3) = 10 in x (@use1) + 100 ) in 
      x (@use2) + y (@use3)
~~~

In this example, we add "comment tags" to each let binding (`tag1`, `tag2` and `tag3`) and to each usage (`use1`, `use2`, `use3`). The tags play no role in the program execution other than giving names to refer to the appropriate definitions and usages of identifiers. 

The purpose of this analysis is to produce a Map that maps usage tags to the let binding tags.

~~~
use1 -> tag3
use2 -> tag1
use3 -> tag2
~~~

This denotes that the usage tagged "use1" corresponds to the let-binding "tag3", "use2" usage corresponds to let binding "tag1" and "use3" usage corresponds to let binding "tag2".




### Abstract Syntax

$$\begin{array}{rcll}
  \Expr & \Rightarrow & \text{Const}(\mathbf{Double}) \\
  & | & \text{Ident}(\mathbf{String}, \color{red}{\mathbf{Tag}} ) \\
  & | & \text{Plus}(\mathbf{Expr}, \mathbf{Expr}) \\
  & | & \text{Mult}(\mathbf{Expr}, \mathbf{Expr})\\
  & | & \text{Let}(\mathbf{Ident}, \color{red}{\mathbf{Tag}}, \mathbf{Expr}, \mathbf{Expr}) \\
  \color{red}{\mathbf{Tag}} & \Rightarrow & \mathbf{String} \\
  \end{array}$$
  
We will modify the abstract syntax as shown by the grammar above. Note that we add a new non-terminal called __Tag__ that is simply another name for a scala String datatype. We add these tags to `Ident` and `Let` which are the places where new identifiers are produced.

Note: 

 - We do not really differentiate use/def tags -- they are all strings that are user defined.
 - All tags must be distinct from one another. This need not be checked.
 

### Part A: Write down Semantic Rules

We will now define the function: `getAllUseDefLinks(e: Expr, env: Map[String, String]): Map[String, String]` that will return a map from the "use tags" strings to the "def tags" strings. 
  - We will use an extra parameter `env` to keep track of a "tag environment", which maps idenfiers to tags. This is similar in one sense to the environment we have seen for `eval` but rather than mapping to a `Value` for each identifier, it maps to a "tag".
  
  


$$\newcommand\semrule[3]{\begin{array}{c} #1 \\ \hline #2 \\ \end{array}(\text{#3})} $$
$$\newcommand\getAllUseDefLinks{\textsf{getAllUseDefLinks}}$$

Here is the rule for a let-binding:

$$\semrule{ \getAllUseDefLinks(\texttt{e1}, \text{env}) = \text{map1},\ \text{env}' = \text{env}\circ [id \mapsto tag],\ \getAllUseDefLinks(\texttt{e2}, \text{env}') = \text{map2} }{ \getAllUseDefLinks(\texttt{Let(id, tag, e1, e2)}, \text{env}) = map1 + map2  }{let-binding}$$ 

  - First compute the use-def mappings for the RHS expression `e1`.
  - Next, update the environment to note that identifier `id` is associated with `tag`.
  - Now, compute the mappings for "in" expression `e2` under the new environment.
  - Conjoin the two maps for `e1` and `e2`.


Here are two rules for a identifiers:


$$\semrule{ \texttt{id} \in \mathsf{domain}(\text{env})}{ \getAllUseDefLinks(\texttt{Ident(id, usetag)}, \text{env}) = \{ \texttt{usetag} \rightarrow \text{env}(\texttt{id}) \} }{ident-defined}$$ 

- If the identifier `id` belongs to the domain of the environment, then return a singleton mapping from the use-tag `usetag` associated with this use of `id` to the tag looked up from the environment.


$$\semrule{ \texttt{id} \not\in \mathsf{domain}(\text{env})}{ \getAllUseDefLinks(\texttt{Ident(id, usetag)}, \text{env}) = \{ \texttt{usetag} \rightarrow  \text{"UNDEFINED"} \} }{ident-undefined}$$ 

Note that we reserve a special tag name `UNDEFINED` (i.e., the string "UNDEFINED") to denote identifiers that are not defined in the current scope.

- If the identifier `id` does not belong to the domain of the environment, then return a singleton mapping from the use-tag `usetag` associated with this use of `id` to the special tag `UNDEFINED`.


### Part (A) : Complete the rule for Plus (5 points).

$$\semrule{ ???_1}{ \getAllUseDefLinks(\texttt{Plus(e1, e2)}, \text{env}) = ???_2 }{plus}$$ 

Write down what should go in for $???_1$ and $???_2$ below.

 - If there are no antecedents just write "blank". 
 - The empty map is represented as $\{\}$.
 - Concatenation of two maps is `map1` and `map2` is denoted `map1 + map2`: the scala operator for this is `++` though.

YOUR ANSWER HERE

## Complete the rule for Const (5 points)

$$\semrule{ ???_1}{ \getAllUseDefLinks(\texttt{Const(f)}, \text{env}) = ???_2 }{const}$$ 

Write down what should go in for $???_1$ and $???_2$ below.

  - If there are no antecedents just write "blank". 
  - The empty map is represented as $\{\}$.
  - Concatenation of two maps is `map1` and `map2` is denoted `map1 + map2`.

YOUR ANSWER HERE

the rule for Plus:

    ???1 = 𝗀𝖾𝗍𝖠𝗅𝗅𝖴𝗌𝖾𝖣𝖾𝖿𝖫𝗂𝗇𝗄𝗌(e1, env) = v1, 𝗀𝖾𝗍𝖠𝗅𝗅𝖴𝗌𝖾𝖣𝖾𝖿𝖫𝗂𝗇𝗄𝗌(e2, env) = v2, v3 = v1 ++ v2
    
    ???2 = v3
    
the rule for const:

    ???1 = 𝗀𝖾𝗍𝖠𝗅𝗅𝖴𝗌𝖾𝖣𝖾𝖿𝖫𝗂𝗇𝗄𝗌(blank)
    
    ???2 = {}

In [7]:
sealed trait Expr
type Tag = String 
type Identifier = String 

case class Const(d: Double) extends Expr
case class Ident(id:Identifier, tag: Tag) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Mult(e1: Expr, e2: Expr) extends Expr 
case class Let(id: Identifier, tag: Tag, e1: Expr, e2: Expr) extends Expr

// Some helpful functions to write test cases -- you can ignore these.

def x(tag:String):Expr = Ident("x",  tag)
def y(tag:String):Expr = Ident("y",  tag)
def z(tag:String):Expr = Ident("z",  tag)
def w(tag:String):Expr = Ident("w", tag)




defined trait Expr
defined type Tag
defined type Identifier
defined class Const
defined class Ident
defined class Plus
defined class Mult
defined class Let
defined function x
defined function y
defined function z
defined function w

In [8]:
def getAllUseDefLinks(e: Expr, env: Map[Identifier, Tag]): Map[Tag, Tag] = 

    e match {
        case Const(_) => { Map.empty }
        case Ident(id, usetag) => {
            if(env.contains(id))
            {
                env + (usetag -> env(id))
            }
            else
                env + (usetag -> "UNDEFINED")
        
                
        }
    
        case Plus(e1, e2) => {
            val v1 = getAllUseDefLinks(e1, env);
            val v2 = getAllUseDefLinks(e2, env);
            val v3 = v1 ++ v2
            return v3
            
        }
    
        case Mult(e1, e2) => {
            val v1 = getAllUseDefLinks(e1, env);
            val v2 = getAllUseDefLinks(e2, env);
            val v3 = v1 ++ v2;
            return v3
           
            
        }
    
        case Let(x, tag, e1, e2) => {
            val v1 = getAllUseDefLinks(e1, env);
            val envP = env + (x -> tag)
            val v2 = getAllUseDefLinks(e2, envP);
            val v3 = v1 ++ v2;
            return v3
        }
      
}


defined function getAllUseDefLinks

In [9]:
/*
 let y@tag2 = x @use0 in 
    let x @tag1 = 10 in 
       x@use1 + x@use2
*/
val e = Let("y", "tag2", x("use0"), Let ("x", "tag1", Const(10), Plus(x("use1"), x("use2"))))
val map = getAllUseDefLinks(e, Map.empty)

assert (map("use0") == "UNDEFINED", s"Test1 failed: use0 must be associated with UNDEFINED. Your code returns ${map("use0")}")
assert (map("use1") == "tag1", s"Test1 failed: use0 must be associated with UNDEFINED. Your code returns ${map("use1")}")
assert (map("use2") == "tag1", s"Test1 failed: use0 must be associated with UNDEFINED. Your code returns ${map("use2")}")

passed(5)


*** Tests Passed (5 points) ***


e: Let = Let(
  id = "y",
  tag = "tag2",
  e1 = Ident(id = "x", tag = "use0"),
  e2 = Let(
    id = "x",
    tag = "tag1",
    e1 = Const(d = 10.0),
    e2 = Plus(
      e1 = Ident(id = "x", tag = "use1"),
      e2 = Ident(id = "x", tag = "use2")
    )
  )
)
map: Map[Tag, Tag] = HashMap(
  "x" -> "tag1",
  "y" -> "tag2",
  "use0" -> "UNDEFINED",
  "use1" -> "tag1",
  "use2" -> "tag1"
)

In [10]:
/* 
let x @tag3 =  x @use0 + 10 in 
   let y@tag1 = 10 in 
      x@use1 * y @use2 */

val e =  Let("x", "tag3", Plus(x("use0"), Const(10.0)), 
            Let ("y", "tag1", Const(10), Mult(x("use1"), y("use2"))))
val map = getAllUseDefLinks(e, Map.empty)
assert (map("use0") == "UNDEFINED", s"Test1 failed: use0 must be associated with UNDEFINED. Your code returns ${map("use0")}")
assert (map("use1") == "tag3", s"Test1 failed: use1 must be associated with UNDEFINED. Your code returns ${map("use1")}")
assert (map("use2") == "tag1", s"Test1 failed: use2 must be associated with tag1. Your code returns ${map("use2")}")

passed(5)


*** Tests Passed (5 points) ***


e: Let = Let(
  id = "x",
  tag = "tag3",
  e1 = Plus(e1 = Ident(id = "x", tag = "use0"), e2 = Const(d = 10.0)),
  e2 = Let(
    id = "y",
    tag = "tag1",
    e1 = Const(d = 10.0),
    e2 = Mult(
      e1 = Ident(id = "x", tag = "use1"),
      e2 = Ident(id = "y", tag = "use2")
    )
  )
)
map: Map[Tag, Tag] = HashMap(
  "x" -> "tag3",
  "y" -> "tag1",
  "use0" -> "UNDEFINED",
  "use1" -> "tag3",
  "use2" -> "tag1"
)

## That's All Folks!